In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import torch
import csv
import kagglehub

# -------------------------
# CONFIGURATION
# -------------------------

DATASET = "darysha/hse-hackathon"
FILE_PATH = "train.csv"   # chemin interne du dataset Kaggle
CSV_OUTPUT = "train_fr.csv"

COLUMNS_TO_TRANSLATE = [
    "cv_status", "localityName", "gender", "positionName",
    "typicalPosition_cv", "skills_cv", "otherCertificates",
    "country", "educationList", "hardSkills_cv", "softSkills_cv",
    "workExperienceList", "scheduleType_cv", "retrainingCapability_cv",
    "businessTrip", "languageKnowledge_cv", "relocation", "innerInfo",
    "education", "vacancyName", "professionalSphereName", "vacancyAddress",
    "busyType_vacancy", "educationRequirements", "hardSkills_vacancy",
    "softSkills_vacancy", "skills_vacancy", "typicalPosition_vacancy",
    "scheduleType_vacancy", "otherVacancyBenefit", "needMedcard",
    "sourceType", "contactPerson", "fullCompanyName", "regionName",
    "positionRequirements", "contactList", "additionalRequirements",
    "qualifications", "responsibilities", "medicalDocument",
    "benefit", "conditions"
]

MODEL_NAME = "Helsinki-NLP/opus-mt-ru-fr"
BATCH_SIZE = 32
MAX_LENGTH = 256

# -------------------------
# LOAD MODEL
# -------------------------

print("🔄 Chargement du modèle de traduction…")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"💻 Device utilisé : {device}")

# -------------------------
# DOWNLOAD & LOAD CSV FROM KAGGLE
# -------------------------

print("📥 Téléchargement du train.csv depuis Kaggle…")
local_file = kagglehub.dataset_download(DATASET, FILE_PATH)

print("📄 Auto-détection du délimiteur…")
with open(local_file, "r", encoding="utf-8") as f:
    sample = f.read(50000)

dialect = csv.Sniffer().sniff(sample)
print(f"✔ Délimiteur détecté : '{dialect.delimiter}'")

print("📄 Lecture robuste du CSV…")
df = pd.read_csv(
    local_file,
    sep=dialect.delimiter,
    engine="python",
    quotechar='"',
    escapechar="\\",
    on_bad_lines="skip"
)

print(f"✔ CSV chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes.")

# -------------------------
# TRANSLATION FUNCTION
# -------------------------

def translate_batch(batch):
    batch = ["" if x is None else str(x) for x in batch]

    inputs = tokenizer(
        batch,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH
    ).to(device)

    outputs = model.generate(**inputs, max_length=MAX_LENGTH)
    return [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]

# -------------------------
# TRANSLATE COLUMNS
# -------------------------

for col in COLUMNS_TO_TRANSLATE:
    if col not in df.columns:
        print(f"⚠️ Colonne absente : {col} → ignorée")
        continue

    print(f"\n🚀 Traduction de la colonne : {col}…")
    texts = df[col].astype(str).fillna("").tolist()
    translated = []

    for i in tqdm(range(0, len(texts), BATCH_SIZE)):
        batch = texts[i:i+BATCH_SIZE]
        translated_batch = translate_batch(batch)
        translated.extend(translated_batch)

    df[col + "_fr"] = translated
    print(f"✔ Colonne {col} traduite → {col}_fr")

# -------------------------
# SAVE RESULT
# -------------------------

df.to_csv(CSV_OUTPUT, index=False, encoding="utf-8")
print(f"\n🎉 Traduction terminée ! Fichier généré : {CSV_OUTPUT}")

#le dataset est téléchargé et le fichier train.csv est lu


🔄 Chargement du modèle…


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


💻 Utilisation du device : cuda
📄 Lecture robuste du CSV…
✔ CSV chargé avec 91 colonnes.
Délimiteur détecté : '|'

🚀 Traduction de la colonne : cv_status…


100%|██████████| 2768/2768 [12:22<00:00,  3.73it/s]


✔ Colonne cv_status traduite → cv_status_fr

🚀 Traduction de la colonne : localityName…


100%|██████████| 2768/2768 [1:24:15<00:00,  1.83s/it]


✔ Colonne localityName traduite → localityName_fr

🚀 Traduction de la colonne : gender…


100%|██████████| 2768/2768 [07:36<00:00,  6.06it/s]


✔ Colonne gender traduite → gender_fr

🚀 Traduction de la colonne : positionName…


 35%|███▍      | 959/2768 [31:15<23:39,  1.27it/s]